In [462]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import csv
import sys,os
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types


# for named entity
from nltk import word_tokenize, pos_tag, ne_chunk, tree, download
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem.porter import *
# for preparing data
download('vader_lexicon')
%matplotlib inline

#!export GOOGLE_APPLICATION_CREDENTIALS='sentimental.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='sentimental.json'

# nltk for natural language processing

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/chitrankdixit/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [463]:
path =r'./playstore_corpus_tsv/' # use your path
allFiles = glob.glob(path + "/*.tsv")

# for out_count, file in enumerate(allFiles):
#     with open(file, newline='') as csvfile:
#         spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
#         #import pdb;pdb.set_trace()
#         for counter, row in enumerate(spamreader):
#             import pdb;pdb.set_trace()
#             print(row)
            
#     with open(file+out_count, 'wt') as csvfile:
#             writer = csv.writer(csvfile, delimiter='', quotechar='|')
#             #writer.writerow(["#"] + anarkali_characteristics)
#             #import pdb;pdb.set_trace()
#             for element in lehenga_characteristics:
#                 #import pdb;pdb.set_trace()
#                 writer.writerow([element["name"], element["count"]])

In [464]:
# get full data


frame = pd.DataFrame()
list_ = []

# datacolumns = [
#     "Package Name",
#     "App Version Code",
#     "Reviewer Language",
#     "Device",
#     "Review Submit Date and Time",
#     "Review Submit Millis Since Epoch",
#     "Review Last Update Date and Time",
#     "Review Last Update Millis Since Epoch",
#     "Star Rating",
#     "Review Title",
#     "Review Text",
#     "Developer Reply Date and Time",
#     "Developer Reply Millis Since Epoch",
#     "Developer Reply Text,Review Link"
# ]

for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0, encoding = "ISO-8859-1", error_bad_lines=False, sep='\t')
    list_.append(df)
    
    
full_data = pd.concat(list_)
full_data.head(5)

cleaned_data = pd.read_csv("pos_neg_score2_class_mix.tsv",index_col=None, header=0, encoding = "ISO-8859-1", error_bad_lines=False, sep='\t')

In [465]:
# remove the NaN 
full_data["Subject"] = full_data["Subject"].fillna('')
full_data["Description"] = full_data["Description"].fillna('')
full_data["Sentiment"] = full_data["sentiment"].fillna(0)
#full_data["tag"] = full_data["tag"].fillna('')
# full_data["Developer Reply Date and Time"] = full_data["Developer Reply Date and Time"].fillna('')
# full_data["Developer Reply Millis Since Epoch"] = full_data["Developer Reply Millis Since Epoch"].fillna('')
# full_data["Developer Reply Text"] = full_data["Developer Reply Text"].fillna('')

# slicing the required columns only
full_data = full_data[["Description", "tag"]]


In [466]:
# clean mail data
#mail_data

In [467]:
#full_data.loc[full_data['sentiment'] == 0].__len__()

In [468]:
# full_data.columns = ["Description","Subject"]
full_data.head()

,Description,tag
0,This is a brilliant concept. Please start for ...,"["" Please start for men's asap""]"
1,Brilliant concept :),['Brilliant concept :)']
2,,[]
3,Brilliant stuff!! Waiting to see more..,['']
4,,[]


In [469]:
# mail_data = mail_data[["subject", "description"]]
# mail_data.columns = ["Subject", "Description"]
# mail_data.head()

# when mail data get cleaned properly then use the below
#full_data = full_data.append(mail_data)

In [470]:
full_data.tail()

,Description,tag
995,Wow! Never knew that it might be this smooth t...,['']
996,A huge thank you for making it so easy and fun...,['']
997,It is very good app,['It is very good app']
998,It is so higher to lease cloths! Facilitates m...,[' So higher and high-quality']
999,It's miles higher manner to lease cloths. I am...,"[""It's miles higher manner to lease cloths""]"


In [471]:
full_data = full_data.loc[full_data['Description'] != '']

full_data.drop_duplicates(inplace=True)
cleaned_data.drop_duplicates(inplace=True)
tags = full_data[["tag"]]
full_data.head()


,Description,tag
0,This is a brilliant concept. Please start for ...,"["" Please start for men's asap""]"
1,Brilliant concept :),['Brilliant concept :)']
3,Brilliant stuff!! Waiting to see more..,['']
5,This idea will be considered as one of the top...,['']
7,No spending on buying apparels for gf... Why t...,[' Why to buy if u can rent it']


In [472]:
len(full_data), len(tags), len(cleaned_data)

(8305, 8305, 8390)

In [473]:
cleaned_data.head()

,Description,Subject,sentiment
0,This is a brilliant concept. Please start for ...,Great product!,1
1,Brilliant concept :),NaN,1
2,Brilliant stuff!! Waiting to see more..,NaN,1
3,This idea will be considered as one of the top...,Excellent,1
4,No spending on buying apparels for gf... Why t...,Brilliant ..this is what we needed!!,0


In [474]:
sid = SentimentIntensityAnalyzer()
client = language.LanguageServiceClient()

# add a named entities to look for
#import pdb;pdb.set_trace()
negation_list = ["No", "Not"]

tech_list = ["order", "placed", "tech", "issue", "coupon", "item", "order placed", "coupon issue", "tech issue", "App crash", "web crash", "ios", 
            "android", "signup issue", "signin issue", "order placed", "order confirmation", "online payment", "hang", "too slow"
            , "cart", "image not matching"]
ops_list = ["delivery", "measurement", "pickup", "refund",  "fitting", "trial", "pickup", "refund", "deposit", "on-time", "delay", "address updation", "return", "cancellation", "exchange",
"payment issues"]
product_list = ["content", "available", "error", "loading", "website not loading", "site not loading", "content not available", "cloth tear", "defective product", "discount codes"]

def get_sentiment_score(item):
    negate = None
    for sentence in item['Description'].split('.'):
        for element in negation_list:
            m = re.findall('^'+element+' \w+', sentence)
            if m:
                break
    if m:
        negate, sentence = sentence[:2],sentence[2:]
    vs = analyzer.polarity_scores(sentence)
    if negate:
        vs['pos'] = 1.0
        vs['neu'] = 0.0
        vs['neg'] = 0.0
    print("{:-<65} {}".format(sentence, str(vs)))
#     sentences = item['Description'].split('.')#.encode('utf-16')
#     #import pdb;pdb.set_trace()
#     #a = b
#     sentiment = None
#     sentence_dict = {}
#     min_score = 100
#     min_text_tag_list = []
#     for text in sentences:
#         document = types.Document(
#             content=text,
#             type=enums.Document.Type.PLAIN_TEXT)

#         # Detects the sentiment of the text
        
#         sentiment = client.analyze_sentiment(document=document).document_sentiment
#         if sentiment.score < min_score:
#             min_score = sentiment.score
#             min_text_tag_list.append(text) 
#         sentence_dict.update({text: sentiment.score})
        
#     sentiment_score= sum(sentence_dict.values()) / len(sentence_dict.values())
#     if sentiment_score <= -0.25 and sentiment_score > -1.0:
#         sentiment = 0
#     elif sentiment_score <= 0.25 and sentiment_score > -0.25:
#         sentiment = 1
#     elif sentiment_score <= 1.00 and sentiment_score > 0.25:
#         sentiment = 2
        
    
    
    
    
    

#     negate = None
#     for element in negation_list:
#         m = re.findall('^'+element+' \w+', sentences)
#         if m:
#             break
#     if m:
#         negate, sentences = sentences[:2],sentences[2:]
#     ss = sid.polarity_scores(sentences)
#     if negate:
#         if ss['neg'] > max(ss['pos'], ss['neu']): 
#             ss['pos'] = 1.0
#             ss['neu'] = 0.0
#             ss['neg'] = 0.0
#         elif ss['pos'] > max(ss['neg'], ss['neu']): 
#             ss['pos'] = 0.0
#             ss['neu'] = 0.0
#             ss['neg'] = 1.0
# #         elif ss['neu'] > max(ss['pos'], ss['neg']):
# #             ss['pos'] = 1.0
# #             ss['neu'] = 0.0
# #             ss['neg'] = 0.0
    
#     negative = ss["neg"]
#     positive = ss["pos"]
#     neutral = ss["neu"]
    #compound = ss["compound"]
    return pd.Series({'sentiment': sentiment, "tag": min_text_tag_list})


# scores = full_data.apply(get_sentiment_score, axis=1)

# final_data = pd.concat([ full_data , scores], axis=1)
len(tags)

8305

In [475]:
# def get_sentiment_class(item):
#     sentiment_class = 0
#     score_list = [item['neutral'], item['negative'], item['positive']]
#     max_val = max(score_list)
    
#     sentiment_class = score_list.index(max_val)
    
#     return pd.Series({'sentiment_class': sentiment_class})


# score_class = final_data.apply(get_sentiment_class, axis=1)

# final_sliced_data = pd.concat([ final_data , score_class], axis=1)
len(cleaned_data), len(tags)
#cleaned_data['tags'] = tags['tag']

#cleaned_data = pd.concat([ cleaned_data , full_data['tag']], axis=1)
cleaned_data = pd.merge(cleaned_data, full_data[["Description", "tag"]])
cleaned_data.head()

,Description,Subject,sentiment,tag
0,This is a brilliant concept. Please start for ...,Great product!,1,"["" Please start for men's asap""]"
1,Brilliant concept :),NaN,1,['Brilliant concept :)']
2,Brilliant stuff!! Waiting to see more..,NaN,1,['']
3,This idea will be considered as one of the top...,Excellent,1,['']
4,No spending on buying apparels for gf... Why t...,Brilliant ..this is what we needed!!,0,[' Why to buy if u can rent it']


In [476]:
# bring the data with probability tech + , tech - , ops + , ops - , product + , product -
import re
tech_list = ["order", "placed", "tech", "issue", "coupon", "item", "order placed", "coupon issue", "tech issue", "app crash", "web crash", "ios", "login" 
            "android", "signup issue", "signin issue", "order placed", "order confirmation", "online payment", "hang", "too slow"
            , "cart", "image not matching"]
ops_list = ["delivery", "measurement", "pickup", "refund",  "fitting", "trial", "pickup", "refund", "deposit", "on-time", "delay", "address updation", "return", "cancellation", "exchange",
"payment issues"]
product_list = ["content", "available", "error", "loading", "website not loading", "site not loading", "content not available", "cloth tear", "defective product", "discount codes"]

def assign_to_team(item):
    team = []
    assign = []
    tags = "".join(list(item['tag']))
    tags = tags[1:-1]
    tags_list = [tags]
    #print(tags_list)
    for sentence in tags_list:
        sentence = sentence.lower()
        #print(sentence)
        for element in tech_list:
            match = re.findall(element, sentence)
            
            if match:
                team.append("tech")
                break
        
        for element in ops_list:
            match = re.findall(element, sentence)
            
            if match:
                team.append("ops")
                break
        
        for element in product_list:
            match = re.findall(element, sentence)
            
            if match:
                team.append("product")
                break
            
        
        print(team)
        assign = assign + list(set(team))
    
    return pd.Series({'team': assign})



team = cleaned_data.apply(assign_to_team, axis=1)

cleaned_data = pd.concat([ cleaned_data , team], axis=1)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['ops']
['tech']
[]
[]
[]
['product']
[]
['product']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tech']
['tech']
['tech']
[]
[]
[]
[]
[]
['tech']
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
['tech']
[]
[]
['tech']
['ops']
[]
[]
['product']
[]
[]
['product']
[]
[]
[]
[]
[]
['tech']
[]
['product']
[]
[]
[]
['product']
[]
[]
[]
[]
[]
['product']
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
['tech']
[]
['ops']
[]
[]
[]
['product']
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
['tech']
[

[]
['product']
['product']
[]
['product']
['ops']
[]
['tech']
[]
[]
['product']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['product']
[]
['product']
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
['product']
[]
[]
[]
['product']
[]
[]
[]
[]
['ops']
[]
[]
[]
['product']
[]
[]
['product']
['ops']
[]
[]
[]
[]
[]
[]
['product']
[]
[]
[]
['ops']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['product']
['product']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['product']
[]
[]
[]
['ops']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['product']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['product']
['product']
[]
[]
['tech']
[]
[]
[]
['product']
['product']
[]
[]
[]
[]
[]
[]
[]
['product']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['product']
[]
[]
[]
[]
[]
['product']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['product']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['ops']
[]
[]
[]
[]
[]
['tech', 'ops']
['product']
['product']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['product']
[]


[]
[]
[]
[]
[]
[]
['product']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['product']
[]
[]
[]
['product']
[]
[]
[]
[]
[]
[]
[]
['product']
[]
[]
[]
[]
['ops']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['product']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['product']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
['ops', 'product']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['product']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['ops']
[]
['ops']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['ops']
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['ops']
['tech']
['ops']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['ops']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
['product']
[]
[]
[]
[]
[]
[]
[]
['product']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['product']
[]
[]
[]
[]
['product']
['tech', 'ops']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['ops']
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['ops']
[]
[]
[]
[]
['tech']
[]
['tech']
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
['tech']
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['product']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
[]
['tech']
['tech']
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tech']
['tech']
[]
[]
['tech']
[]
['tech']
[]
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tech']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]

In [477]:
print(cleaned_data.loc[cleaned_data['sentiment'] == 0].__len__())
print(cleaned_data.loc[cleaned_data['sentiment'] == 1].__len__())

981
7409


In [478]:
assign = []
team = []
for sentence in cleaned_data['tag'].tolist():
    sentence = sentence.lower()
    for element in tech_list:
        match = re.findall(element, sentence)

        if match:
            team.append("tech")
            break

    for element in ops_list:
        match = re.findall(element, sentence)

        if match:
            team.append("ops")
            break

    for element in product_list:
        match = re.findall(element, sentence)

        if match:
            team.append("product")
            break



    assign.append(list(set(team)))

print(assign)

cleaned_data.head()

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech'], ['tech', 'ops'], ['tech', 'ops'], ['tech', 'ops'], ['tech', 'ops'], ['tech', 'ops'], ['tech', 'product', 'ops'], ['tech', 'product', 'ops'], ['tech', 'product', 'ops'], ['tech', 'product', 'ops'], ['tech', 'product', 'ops'], ['tech', 'product', 'ops'], ['tech', 'product', 'ops'], ['tech', 'product', 'ops'], ['tech', 'product', 'ops'], ['tech', 'product', 'ops'], ['tech', 'product', 'ops'], ['tech', 'product', 'ops'], ['tech', 'product', 'ops'], ['tech', 'product', 'ops'], ['tech', 'prod

,Description,Subject,sentiment,tag,team
0,This is a brilliant concept. Please start for ...,Great product!,1,"["" Please start for men's asap""]",[]
1,Brilliant concept :),NaN,1,['Brilliant concept :)'],[]
2,Brilliant stuff!! Waiting to see more..,NaN,1,[''],[]
3,This idea will be considered as one of the top...,Excellent,1,[''],[]
4,No spending on buying apparels for gf... Why t...,Brilliant ..this is what we needed!!,0,[' Why to buy if u can rent it'],[]


In [479]:
negative_data = cleaned_data.loc[(cleaned_data['sentiment'] == 0)][:981]
positive_data = cleaned_data.loc[(cleaned_data['sentiment'] == 1)][:981]

cleaned_data = positive_data.append(negative_data)

In [480]:
cleaned_data["sentiment"]

0       1
1       1
2       1
3       1
5       1
6       1
7       1
10      1
12      1
13      1
14      1
16      1
17      1
18      1
19      1
21      1
23      1
25      1
26      1
28      1
29      1
30      1
32      1
33      1
34      1
35      1
36      1
37      1
38      1
39      1
       ..
8336    0
8338    0
8339    0
8340    0
8342    0
8344    0
8346    0
8347    0
8348    0
8351    0
8355    0
8358    0
8359    0
8360    0
8361    0
8364    0
8365    0
8366    0
8367    0
8368    0
8369    0
8372    0
8374    0
8376    0
8377    0
8378    0
8382    0
8383    0
8384    0
8388    0
Name: sentiment, Length: 1962, dtype: int64

In [481]:
# generate csv file
# final_sliced_data.to_csv("text_data.tsv", sep='\t')

In [482]:
# final_sliced_data.to_csv('pos_neg_score.csv', index = False)

In [483]:
print(cleaned_data.loc[cleaned_data['sentiment'] == 0].__len__())
print(cleaned_data.loc[cleaned_data['sentiment'] == 1].__len__())
#print(cleaned_data.loc[cleaned_data['sentiment'] == 2].__len__())

981
981


In [484]:
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy


def format_sentence(sent):
    return({word: True for word in word_tokenize(sent)})

In [485]:
stemmer = PorterStemmer()

class WordWeightAssign:
    """
        This is the class to assign the word and its weight
    """
    def __init__(self, data):
        self.data = data
        
    def collect_data(self):
        self.unique_words = []
        for element in self.data:
            for item in element:
                #item = item.lower()
                item = stemmer.stem(item)
                self.unique_words.append(item)
        self.unique_words = list(set(self.unique_words))
        return self.unique_words
    
    def score_words(self):
        review_word_scores = []
        #import pdb;pdb.set_trace()
        for item in self.data:
            local_scores = np.zeros((4640,), dtype=np.int)
            for element in item:
                element = stemmer.stem(element)
                current_index = self.unique_words.index(element)
                if current_index:
                    local_scores[current_index] += 1
            review_word_scores.append(local_scores)
        #import pdb;pdb.set_trace()
        return review_word_scores

In [486]:
w = WordWeightAssign(cleaned_data['Description'].str.split(' ').tolist())
print(w.collect_data().__len__())
print(w.score_words().__len__())
#print(w.score_words())

4640
1962


In [487]:
cleaned_data.head()

,Description,Subject,sentiment,tag,team
0,This is a brilliant concept. Please start for ...,Great product!,1,"["" Please start for men's asap""]",[]
1,Brilliant concept :),NaN,1,['Brilliant concept :)'],[]
2,Brilliant stuff!! Waiting to see more..,NaN,1,[''],[]
3,This idea will be considered as one of the top...,Excellent,1,[''],[]
5,Brilliant idea!! Adorably splendid dresses at ...,NaN,1,"['', 'jwellery']",[]


In [489]:
labels = cleaned_data["sentiment"].values;

#import pdb;pdb.set_trace()  
kf = StratifiedKFold(n_splits=4)


totalNBV = 0

totalMatNBV = np.zeros((2,2));


totalNBV_tfidf = 0

totalMatNBV_tfidf = np.zeros((2,2));

# tr, te = kf.split(sliced_data['Review Text'], labels)
# td = []
# for i in train_index:
#     td.append(sliced_data['Review Text'][i])
#import pdb;pdb.set_trace()
review_text = cleaned_data['Description'].tolist()

# break words
review_words = []

# for item in review_text:
#     review_words += item.split(" ")
counter = 0
for train_index, test_index in kf.split(review_text, labels):
    #import pdb;pdb.set_trace()
    #print(train_index.__len__(), test_index.__len__())
    X_train = [review_text[i] for i in train_index]
    X_test = [review_text[i] for i in test_index]
    
    y_train, y_test = labels[train_index], labels[test_index]
    
    vectorizer = TfidfVectorizer(min_df=5, max_df = 0.8, sublinear_tf=True, use_idf=False, stop_words='english')
    train_corpus_tf_idf = vectorizer.fit_transform(X_train) 
    test_corpus_tf_idf = vectorizer.transform(X_test)
    
    splitted_x_train = [item.split(' ') for item in X_train]
    w = WordWeightAssign(splitted_x_train)
    w.collect_data()
    w.score_words()
    train_corpus_scores = w.score_words()
    
    splitted_x_test = [item.split(' ') for item in X_test]
    w = WordWeightAssign(splitted_x_test)
    w.collect_data()
    w.score_words()
    test_corpus_scores = w.score_words()
    
    
    nb = MultinomialNB()
    nb.fit(train_corpus_scores, y_train)
    result1 = nb.predict(test_corpus_scores)
    test = nb.score(test_corpus_scores, y_test)
    print(test)
    
    nbtfidf = MultinomialNB()
    nbtfidf.fit(train_corpus_tf_idf, y_train)
    result2 = nbtfidf.predict(test_corpus_tf_idf)
    test = nbtfidf.score(test_corpus_tf_idf, y_test)
    print(test)
    
    totalMatNBV = totalMatNBV + confusion_matrix(y_test, result1)
    totalNBV = totalNBV + sum(y_test==result1)
    
    totalMatNBV_tfidf = totalMatNBV_tfidf + confusion_matrix(y_test, result2)
    totalNBV_tfidf = totalNBV_tfidf + sum(y_test==result2)
    
    counter += 1
    prev = cleaned_data.iloc[test_index, :]
    test = pd.DataFrame( { 'Review Text': X_test , 'Sentiment': result1, "Team": prev['team']})
    test.to_csv( 'sentiment_prediction_custom_'+str(counter)+'.csv' , index = False )
    
    test = pd.DataFrame( { 'Review Text': X_test , 'Sentiment': result2, "Team": prev['team']})
    test.to_csv( 'sentiment_prediction_tfidf_'+str(counter)+'.csv' , index = False )
    
    
print(totalMatNBV, totalNBV/1962.0, totalMatNBV_tfidf, totalNBV_tfidf/1962.0)
    

0.461382113821
0.778455284553
0.561224489796
0.734693877551
0.469387755102
0.851020408163
0.465306122449
0.85306122449
[[ 464.  517.]
 [ 485.  496.]] 0.489296636086 [[ 768.  213.]
 [ 171.  810.]] 0.804281345566


In [ ]:
cleaned_data.